In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import os
import subprocess
import numpy as np
import mediapipe as mp
import csv
import pandas as pd
import openpyxl
from openpyxl.chart import LineChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ============================================================
#   FUNCIÓN DE PREPROCESAMIENTO (NO SE TOCA)
# ============================================================
def preprocess_frame(frame):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb = cv2.bilateralFilter(rgb, 7, 60, 60)

    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge([l, a, b])
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    edges = cv2.magnitude(sobelx, sobely)
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX).astype("uint8")
    edges_color = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    rgb = cv2.addWeighted(rgb, 1.0, edges_color, 0.15, 0)

    kernel_sharpen = np.array(
        [[-1, -1, -1],
         [-1,  9, -1],
         [-1, -1, -1]]
    )
    rgb = cv2.filter2D(rgb, -1, kernel_sharpen)

    return rgb

In [ ]:
# ============================================================
#   RUTAS Y CARPETAS
# ============================================================
input_folder = "/content/drive/MyDrive/PESO MUERTO"
output_folder_videos = "/content/drive/MyDrive/PESO MUERTO_POSTURA"
output_folder_csv = "/content/drive/MyDrive/CURL DE BICEPS_CSV"
output_global_csv = "/content/drive/MyDrive/CURL DE BICEPS DATOS/_POSTURA_CURL_DE_BICEPS_GLOBAL.csv"
output_excel_stats = "/content/drive/MyDrive/CURL DE BICEPS DATOS/POSTURA_CURL_DE_BICEPS_ESTADISTICAS.xlsx"

os.makedirs(output_folder_videos, exist_ok=True)
os.makedirs(output_folder_csv, exist_ok=True)

videos = [f for f in os.listdir(input_folder) if f.lower().endswith(".mp4")]
print(f"\n🎬 Se encontraron {len(videos)} videos.\n")


🎬 Se encontraron 280 videos.



In [ ]:
# ============================================================
#   CONTENEDOR PARA CSV GLOBAL
# ============================================================
global_data = []

In [ ]:
# ============================================================
#   PROCESAMIENTO POR VIDEO
# ============================================================
for video in videos:

    input_path = os.path.join(input_folder, video)
    tmp_opt = "/content/tmp_opt.mp4"
    tmp_pre = "/content/tmp_pre.mp4"
    tmp_skeleton = "/content/tmp_skeleton_raw.mp4"

    output_video = os.path.join(
        output_folder_videos,
        video.replace(".mp4", "_Postura.mp4")
    )

    output_csv = os.path.join(
        output_folder_csv,
        video.replace(".mp4", ".csv")
    )

    print(f"🔧 Procesando: {video}")



    # ============================================================
    #   1. OPTIMIZAR EL VIDEO ORIGINAL
    # ============================================================
    cmd_opt = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-an",
        "-r", "30",
        "-b:v", "900k",
        "-c:v", "libx264",
        "-preset", "fast",
        tmp_opt
    ]
    subprocess.run(cmd_opt)



    # ============================================================
    #   2. PREPROCESAMIENTO FRAME A FRAME
    # ============================================================
    cap = cv2.VideoCapture(tmp_opt)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_pre = cv2.VideoWriter(tmp_pre, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed = preprocess_frame(frame)
        processed_bgr = cv2.cvtColor(processed, cv2.COLOR_RGB2BGR)
        out_pre.write(processed_bgr)

    cap.release()
    out_pre.release()



    # ============================================================
    #   3. MEDIAPIPE – GENERAR ESQUELETO + CSV DE LANDMARKS
    # ============================================================
    cap = cv2.VideoCapture(tmp_pre)
    out_skeleton = cv2.VideoWriter(tmp_skeleton, fourcc, fps, (w, h))

    csv_file = open(output_csv, "w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["video", "frame", "landmark_id", "x", "y", "z", "visibility"])

    frame_idx = 0
    fail_frames = 0

    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(img_rgb)

            black = np.zeros((h, w, 3), dtype=np.uint8)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS
                )

                for idx, lm in enumerate(results.pose_landmarks.landmark):
                    row = [video, frame_idx, idx, lm.x, lm.y, lm.z, lm.visibility]
                    csv_writer.writerow(row)
                    global_data.append(row)

            else:
                fail_frames += 1

            out_skeleton.write(black)
            frame_idx += 1

    cap.release()
    out_skeleton.release()
    csv_file.close()



    # ============================================================
    #   4. RECODIFICAR VIDEO FINAL
    # ============================================================
    cmd_final = [
        "ffmpeg", "-y",
        "-i", tmp_skeleton,
        "-an",
        "-r", "30",
        "-b:v", "900k",
        "-c:v", "libx264",
        "-preset", "fast",
        output_video
    ]
    subprocess.run(cmd_final)

    print(f"📄 CSV listo: {output_csv}")
    print(f"🎥 Video listo: {output_video}\n")

🔧 Procesando: 1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_1.mp4
📄 CSV listo: /content/drive/MyDrive/CURL DE BICEPS_CSV/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_1.csv
🎥 Video listo: /content/drive/MyDrive/CURL DE BICEPS_POSTURA/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_1_Postura.mp4

🔧 Procesando: 1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_2.mp4
📄 CSV listo: /content/drive/MyDrive/CURL DE BICEPS_CSV/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_2.csv
🎥 Video listo: /content/drive/MyDrive/CURL DE BICEPS_POSTURA/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_2_Postura.mp4

🔧 Procesando: 1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_3.mp4
📄 CSV listo: /content/drive/MyDrive/CURL DE BICEPS_CSV/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_3.csv
🎥 Video listo: /content/drive/MyDrive/CURL DE BICEPS_POSTURA/1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_3_Postura.mp4

🔧 Procesando: 1_AchipizDaniel_CurlBiceps_Frontal_Bien_Rep_4.mp4
📄 CSV listo: /content/drive/MyDrive/CURL DE BICEPS_CSV/1_AchipizDanie

In [ ]:
# ============================================================
#   5. GUARDAR CSV GLOBAL
# ============================================================
df_global = pd.DataFrame(global_data, columns=[
    "video", "frame", "landmark_id", "x", "y", "z", "visibility"
])
df_global.to_csv(output_global_csv, index=False)
print(f"📌 CSV GLOBAL guardado en: {output_global_csv}")

📌 CSV GLOBAL guardado en: /content/drive/MyDrive/CURL DE BICEPS DATOS/_POSTURA_CURL_DE_BICEPS_GLOBAL.csv


In [ ]:
# ============================================================
#   6. GENERAR EXCEL DE ESTADÍSTICAS (con gráficos)
# ============================================================
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Estadísticas"

stats = []

for landmark_id in range(33):
    df_lm = df_global[df_global["landmark_id"] == landmark_id]

    if df_lm.empty:
        continue

    stats.append([
        landmark_id,
        df_lm["x"].mean(), df_lm["y"].mean(), df_lm["z"].mean(),
        df_lm["x"].std(), df_lm["y"].std(), df_lm["z"].std(),
        df_lm["x"].min(), df_lm["y"].min(), df_lm["z"].min(),
        df_lm["x"].max(), df_lm["y"].max(), df_lm["z"].max(),
        df_lm["visibility"].mean(),
        (df_lm["visibility"] > 0.7).mean(),
        df_lm["frame"].nunique(),
        df_lm[df_lm["visibility"] < 0.01]["frame"].nunique()
    ])

df_stats = pd.DataFrame(stats, columns=[
    "landmark_id",
    "mean_x","mean_y","mean_z",
    "std_x","std_y","std_z",
    "min_x","min_y","min_z",
    "max_x","max_y","max_z",
    "visibility_mean",
    "visibility_gt_0.7",
    "frames_total",
    "frames_failed"
])

# Añadir al Excel
for r in dataframe_to_rows(df_stats, index=False, header=True):
    ws.append(r)

# Crear gráfico
chart = LineChart()
chart.title = "Visibilidad promedio por punto"
chart.y_axis.title = "Visibilidad"
chart.x_axis.title = "Landmark ID"

data = Reference(ws, min_col=14, min_row=1, max_row=len(df_stats)+1)
cats = Reference(ws, min_col=1, min_row=2, max_row=len(df_stats)+1)

chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)

ws.add_chart(chart, "Q2")

# Guardar Excel
wb.save(output_excel_stats)

print(f"📘 Excel con estadísticas generado: {output_excel_stats}")

print("\n🎉 PROCESO COMPLETO")


📘 Excel con estadísticas generado: /content/drive/MyDrive/CURL DE BICEPS DATOS/POSTURA_CURL_DE_BICEPS_ESTADISTICAS.xlsx

🎉 PROCESO COMPLETO
